In [2]:
! git clone https://github.com/sp4s-s/ugen-v01/

Cloning into 'ugen-v01'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 21.96 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
%%capture
! pip install huggingface_hub accelerate

In [4]:
%cd ugen-v01

/content/ugen-v01


In [7]:
! python download.py

amber-v1.zip: 100% 163M/163M [00:02<00:00, 77.9MB/s]
Extracted JPGs from amber-v1.zip
klee-v1.zip: 100% 109M/109M [00:01<00:00, 59.6MB/s]
Extracted JPGs from klee-v1.zip
nahida-v1.0.zip: 100% 158M/158M [00:01<00:00, 79.0MB/s]
Extracted JPGs from nahida-v1.0.zip
rushia-v2.zip: 100% 237M/237M [00:03<00:00, 76.1MB/s]
Extracted JPGs from rushia-v2.zip
shiroko-v1.zip: 100% 390M/390M [00:06<00:00, 62.6MB/s]
Extracted JPGs from shiroko-v1.zip
yoimiya-v1.zip: 100% 281M/281M [00:04<00:00, 70.3MB/s]
Extracted JPGs from yoimiya-v1.zip
collei-v1.zip: 100% 179M/179M [00:02<00:00, 71.3MB/s]
Extracted JPGs from collei-v1.zip
ganyu-v1.zip: 100% 256M/256M [00:03<00:00, 78.8MB/s]
Extracted JPGs from ganyu-v1.zip
hifumi-v1.zip: 100% 97.3M/97.3M [00:01<00:00, 65.4MB/s]
Extracted JPGs from hifumi-v1.zip
youmu-v1.zip: 100% 125M/125M [00:03<00:00, 37.8MB/s]
Extracted JPGs from youmu-v1.zip
azusa-blue-archive-v1.zip: 100% 305M/305M [00:04<00:00, 74.4MB/s]
Extracted JPGs from azusa-blue-archive-v1.zip
Failed o

In [ ]:
! python main.py

2025-05-04 17:43:20.855899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746380600.876244    3510 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746380600.882468    3510 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-04 17:43:20.902727: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Currently logged in as: pigsty (pigsty-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relog

In [9]:
%%writefile main.py
import torch, os, wandb
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split, DataLoader
from accelerate import Accelerator
from unet import UNet
from dataset import ImageNoiseDataset
from utils import save_checkpoint, log_images


data_path = "data"
epochs = 50
bs = 32
lr = 5e-5
checkpoint_dir = "checkpoints"

wandb.init(project="unet-image-gen")
writer = SummaryWriter("runs/unet_image_gen")
accelerator = Accelerator()

full = ImageNoiseDataset(data_path)
train_ds, val_ds = random_split(full, [int(0.9*len(full)), len(full)-int(0.9*len(full))])
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=bs)


model = UNet()
opt = torch.optim.Adam(model.parameters(), lr=lr)

model, opt, train_dl, val_dl = accelerator.prepare(model, opt, train_dl, val_dl)

for epoch in range(epochs):
    model.train()
    for step, (noise, target) in enumerate(train_dl):
        pred = model(noise)
        loss = torch.nn.functional.mse_loss(pred, target)
        accelerator.backward(loss)
        opt.step()
        opt.zero_grad()

        if step % 10 == 0 and accelerator.is_main_process:
            wandb.log({"loss": loss.item()})
            writer.add_scalar("Loss/train", loss.item(), epoch * len(train_dl) + step)

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for noise, target in val_dl:
            pred = model(noise)
            val_loss += torch.nn.functional.mse_loss(pred, target).item()
        val_loss /= len(val_dl)

        if accelerator.is_main_process:
            wandb.log({"val_loss": val_loss})
            writer.add_scalar("Loss/val", val_loss, epoch)
            save_checkpoint(model, f"{checkpoint_dir}/unet_epoch_{epoch}.pt")
            log_images(noise[:4], pred[:4], target[:4], writer, epoch)

wandb.finish()

Overwriting main.py


In [ ]:
! python inference.py